In [ ]:
import copy, sys
import logging
logger = logging.getLogger()
logging.getLogger().handlers[:] = []
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format='%(levelname)s: %(message)s',
    force=True
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
def show_grid(grid):
    res = ""
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if grid[y][x] == 0:
                res += "| "
            elif grid[y][x] < 0:
                res += "|#"
            else:
                res += "|" + str(grid[y][x])
        res += "|\n"
    print(res)

In [ ]:
def show_plans(plans, original_grid):
    for time in range(len(plans[0])):
        grid = copy.deepcopy(original_grid)
        for i, plan in enumerate(plans):
            x, y = plan[time]
            if grid[x][y] < 0:
                logger.error(f"Hit obstacle in plan {i} at time {time}")
            grid[x][y] = i + 1  # Mark the grid with plan index + 1 to differentiate
        show_grid(grid)

# Old version - timed flexibility

In [ ]:
def wait_time_partial_plans(trajectory):
    plan_length = len(trajectory)
    wait_times = [0 for _ in range(plan_length)]
    travel_times = [0 for _ in range(plan_length)]
    subplans = []
    i = 0
    while i < plan_length - 1:
        if trajectory[i] == trajectory[i+1]:
            duration = plan_length - 1
            for j in range(i+1, plan_length):
                if trajectory[j] != trajectory[j-1]:
                    duration = max(j-1, i+1)
                    break
            # TODO this allows flexibility on the wait actions
            for k in range(i, duration + 1):
                wait_times[k] = duration - k
            logger.info(f"[Computing wait times] Waiting from time {i} up and till time {j} next time step {duration}")
            i = duration
        else:
            # Gives the number of states that the agent waits
            duration = plan_length - 1
            for j in range(i+1, plan_length):
                if trajectory[j] == trajectory[j-1]:
                    duration = max(j-1, i+1)
                    break
            subplans.append(trajectory[i:j])
            for k in range(i, duration + 1):
                travel_times[k] = duration - k
            logger.info(f"[Computing travel times] Found subplan from time {i} till time {j}: {trajectory[i:j]} next time step {duration}")
            i = duration
    return wait_times, travel_times, subplans

def print_wait_times_and_subplans(trajectories):
    for i, t in enumerate(trajectories):
        res = wait_time_partial_plans(t)
        print(f"Agent {i}: waiting time steps {res[0]} and subplans {res[1]}")

In [ ]:
def get_flex(trajectories):
    plan_length = len(trajectories[0])
    num_agents = len(trajectories)
    flexibility = []
    for agent in range(num_agents):
        wait_times, travel_times, _ = wait_time_partial_plans(trajectories[agent])
        flexibility.append([0 for _ in range(plan_length)])
        # Last point in plan (plan_length-1) does not have any flexibility left
        for timeVar in range(plan_length-2, -1, -1):
            # Maximum flexibility depends on waiting time after this point
            flex = max(wait_times[timeVar:])
            conflict = False
            for other_agent in range(num_agents):
                if agent != other_agent:
                    if trajectories[agent][timeVar] in trajectories[other_agent][timeVar+1:]:
                        index = timeVar + 1 + trajectories[other_agent][timeVar+1:].index(trajectories[agent][timeVar])
                        logger.info(f"[Found conflict] Agent {agent} at time {timeVar} has initial wait time {flex} and other agent {other_agent} crosses node {trajectories[agent][timeVar]} at time {index}")
                        # Flex was initial flexibility of wait time, but is at most the time until other agent reaches this node
                        flex = min(flex, index - timeVar - 1)
                        conflict = True
            logger.info(f"[Compute flexibility] Flexibility of agent {agent} at time {timeVar} is {flex} and flexibility at next time step is {flexibility[agent][timeVar+1] if timeVar < plan_length-2 else flex}")
            if flexibility[agent][timeVar+1] < 0:
                logger.warning(f"[Set flexibility] Agent {agent} at next time {timeVar+1} has a conflict, so cannot have flexibility at current time {timeVar}")
                # Conflict detected afterward
                # TODO if the agent wait again
                flexibility[agent][timeVar] = -1
            elif conflict:
                if flex > 0:
                    logger.info(f"[Set flexibility] Conflict detected in future but still a positive flexibility of {flex} found for agent {agent} at time {timeVar}")
                    flexibility[agent][timeVar] = flex
                else:
                    flexibility[agent][timeVar] = -1
            elif timeVar == plan_length - 1:
                logger.warning(f"[Set flexibility] Agent {agent} at time {timeVar} is at end of plan so has no flexibility")
                flexibility[agent][timeVar] = flex
            else:
                flexibility[agent][timeVar] = max(flex, flexibility[agent][timeVar+1])
        for time in range(plan_length-1, -1, -1):
            pass
            # # If there are no more moves after this point, then there is no flexibility
            # if sum(travel_times[time:]) == 0:
            #     logger.info(f"Flexibility for agent {agent} at {time} is 0 because the agent does not travel anymore after this time")
            #     flexibility[agent][time] = 0
    flexibility = [[x if x >= 0 else 0 for x in f] for f in flexibility]
    return flexibility

In [ ]:
def print_flexibility(trajectories, print_trajectories=False):
    plan_length = len(trajectories[0])
    if print_trajectories:
        for i, t in enumerate(trajectories):
            print(f"    Agent {i} has initial trajectory {t}")
    flex = get_flex(trajectories)
    if print_trajectories:
        for i, agent_flex in enumerate(flex):
            for t, f in enumerate(agent_flex):
                if f > 0:
                    # TODO use the actual waiting points
                    new_trajectory = trajectories[i][:t] + [trajectories[i][t]] * f + trajectories[i][t:plan_length - f]
                    print(f"Agent {i} at time {t} can delay up to {f} time steps")
                    for j in range(len(trajectories)):
                        if i == j:
                            print(f"  New trajectory agent {j}: {new_trajectory}")
                        else:
                            print(f"  Old trajectory agent {j}: {trajectories[j]}")
                            for time in range(plan_length):
                                if trajectories[j][time] == new_trajectory[time]:
                                    logger.error(f"Agents {i} and {j} using flexibility of agent {i} both occupy location {trajectories[j][time]} at time {time}")
    return flex

# 

# Tests


In [ ]:
logger.setLevel(logging.WARN)

In [ ]:
def test(flexibility, expected):
    if flexibility != expected:
        logger.error(f"Expected: {expected}")
        logger.error(f"Received: {flexibility}")
        logger.error("ERROR: Flexibility is incorrect")
        raise ValueError("Flexibility is incorrect")
    else:
        print(f"CORRECT flexibility:")
        for i, f in enumerate(flexibility):
            print(f"  Agent {i} has flexibility {f}")

In [ ]:
# Scenario 1 - grid size 4
trajectories = [
    [(0, 0), (1, 0), (2, 0), (3, 0), (3, 0)],
    [(0, 1), (1, 1), (2, 1), (2, 0), (2, 0)],
    [(0, 2), (1, 2), (2, 2), (2, 1), (2, 1)]
]
flexibility = print_flexibility(trajectories, True)
expected_flexibility = [
    [0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 1, 1, 0],
]
test(flexibility, expected_flexibility)

In [ ]:
# Scenario 2 - corridor
grid = [
    [0, -1, -1, 0],
    [0, 0, 0, 0],
    [0, -1, -1, 0],
]
trajectories = [
    [(0, 0), (0, 0), (0, 0), (0, 0), (1, 0), (1, 1)],
    [(1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0)]   
]
# show_plans(plans, grid)
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [3, 2, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0]
]
test(flexibility, expected_flexibility)
print_wait_times_and_subplans(trajectories)

In [ ]:
# Scenario corridor paper
grid = [
    [0, -1, -1, -1, -1, -1, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, -1, -1, -1, -1, -1, 0],
]
trajectories = [
    [(0, 0), (0, 0), (0, 0), (1 , 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6), (0, 6)],
    [(2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (2, 6), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0)]
]
# TODO arriving at goal node before goal time is flexibility - goal time explicit
# TODO flexibility at end is still allowed - should decrease to 0 in last time step
# show_plans(trajectories, grid)
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [0 for _ in range(10)] + [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
    [9, 8, 7, 6, 5] + [4 for _ in range(13)] + [3, 2, 1, 0]
]
test(flexibility, expected_flexibility)
# print_wait_times_and_subplans(trajectories)

In [ ]:
# Scenario three agents same node - grid size 3
# logger.setLevel(logging.INFO)
trajectories = [
    [(0, 0), (1, 0), (1, 1), (2, 1), (2, 2)],
    [(0, 1), (1, 1), (2, 1), (2, 0), (2, 0)],
    [(0, 2), (1, 2), (1, 2), (1, 1), (2, 1)]
]
show_plans(trajectories, [[0 for _ in range(3)] for _ in range(3)])
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0],
]
test(flexibility, expected_flexibility)

In [ ]:
# Scenario double corridor
logger.setLevel(logging.WARN)
grid = [
    [0, -1, -1, -1, 0, -1, -1, -1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, -1, -1, -1, 0, -1, -1, -1, 0],
]
# show_grid(grid)
trajectories = [
    [(0, 0), (0, 0), (0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (0, 4), (0, 4), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (0, 8), (0, 8), (0, 8), (0, 8), (0, 8), (0, 8), (0, 8), (0, 8)],
    [(2, 4), (2, 4), (2, 4), (2, 4), (2, 4), (2, 4), (2, 4), (2, 4), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0), (2, 0)],
    [(2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (2, 8), (1, 8), (1, 7), (1, 6), (1, 5), (1, 4), (2, 4), (2, 4), (2, 4)]
]
# show_plans(trajectories, grid)
flexibility = print_flexibility(trajectories)
expected_flexibility = [
    [0 for _ in range(len(trajectories[0]))],
    [9 for _ in range(8)] + [1] + [9 for _ in range(4)] + [0 for _ in range(10)],
    [14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3] + [2 for _ in range(8)] + [0 for _ in range(3)]
]
test(flexibility, expected_flexibility)